In [29]:
import numpy as np
import polars
import pandas as pd
from groupby_lib import GroupBy, crosstab, install_groupby_fast
install_groupby_fast()

✅ groupby-lib patches installed methods installed!
   Use df.groupby_fast() and series.groupby_fast() for optimized performance


In [2]:
N = 20_000_000
df = pd.DataFrame(dict(
    floats=np.random.rand(N), 
    ints=np.random.randint(0, 1000, N),
))
df["categorical"] = pd.Categorical.from_codes(df.ints % 10, list("qwertyuiop"))
df_pl = polars.DataFrame(df)

### The GroupBy Class & groupby_fast monkey patch

#### Two ways of utilizing the functionality 

In [3]:
from groupby_lib import GroupBy
from groupby_lib import install_groupby_fast
install_groupby_fast()

✅ groupby-lib patches installed methods installed!
   Use df.groupby_fast() and series.groupby_fast() for optimized performance


#### ***~15x*** faster when grouping by a categorical 4-5x vs. Polars)

In [5]:
%timeit -n 1 df.groupby("categorical", observed=True).sum()   # verbosity to avoid warnings
%timeit -n 1 df_pl.group_by("categorical").sum()
%timeit -n 1 GroupBy(df.categorical).sum(df); 
%timeit -n 1 GroupBy.sum(df.categorical, df); 
%timeit -n 1 df.groupby_fast("categorical").sum()

146 ms ± 20.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
34.3 ms ± 785 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
9.51 ms ± 1.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
9.83 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
10.2 ms ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### ***~3x*** faster when grouping by integers column (randomly distributed, 1000 uniques)

In [7]:
%timeit -n 1 df.groupby("ints", observed=True).mean(numeric_only=True);
%timeit -n 1 df_pl.group_by("ints").mean()
%timeit -n 1 df.groupby_fast("ints").mean();

111 ms ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
68.2 ms ± 4.31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
32.6 ms ± 802 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### ***~3-4x*** faster aggregating already grouped data

##### 3-8x Faster on 2 columns

In [8]:
for key in ["ints", "categorical"]:
    print(key)
    for gb in [
        df.groupby(key, observed=True),
        df.groupby_fast(key)
    ]:
        gb[["floats", "ints"]].mean(); # this is to pay the one-off setup for Pandas groupby
        %timeit -n 1 gb[["floats", "ints"]].mean();
    print()

ints
77 ms ± 3.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
24.2 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

categorical
82.4 ms ± 3.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
9.55 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)



##### 2.5 - 5x faster on 10 columns

In [9]:
wide_df = pd.DataFrame({i: np.random.rand(N // 1) for i in range(10)})
for key in ["ints", "categorical"]:
    print(key)
    key = df[key][:len(wide_df)]
    for gb in [
        wide_df.groupby(key, observed=True),
        wide_df.groupby_fast(key),
    ]:
        gb.mean(); # this is to pay the one-off setup for Pandas groupby
        %timeit -n 1 gb.mean();
    print()

ints
265 ms ± 9.99 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
104 ms ± 2.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

categorical
169 ms ± 8.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
54.3 ms ± 4.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)



#### In-line filtering/masking

#### 6-10x faster with a row mask (with 80% positive rate)

In [11]:
mask = df.floats.between( *df.floats.quantile([.1, .9]))
for key in ["ints", "categorical"]:
    print(f"{key} grouper")
    %timeit -n 1 df.loc[mask].groupby(key, observed=True).mean(numeric_only=True)
    %timeit -n 1 df.groupby_fast(key).mean(mask=mask)
    print()

ints grouper
209 ms ± 18.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
36.6 ms ± 11.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

categorical grouper
250 ms ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
26.6 ms ± 745 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)



####  Multi-Key and Margins

In [12]:
multi_key = ["categorical", df.ints % 3]

In [14]:
%timeit -n 1 df.groupby(multi_key, observed=True).sum(numeric_only=True)
%timeit -n 1 df.groupby_fast(multi_key).sum()

386 ms ± 24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
112 ms ± 6.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
df.groupby_fast(multi_key).sum(
    margins=True, mask=df.categorical.isin(["q", "p"])
).style.format(precision=2)

####  Rolling / EMA functions ~10 / 40x faster*

In [16]:
df_small = df.iloc[:N // 4]

In [17]:
# 10x faster with same behaviour
%timeit -n 1 df_small.groupby("categorical", observed=True).rolling(5, min_periods=1).sum()
%timeit -n 1 df_small.groupby_fast("categorical").rolling(5, min_periods=1).sum(index_by_groups=True)
a = df_small.groupby("categorical", observed=True).rolling(5, min_periods=1).sum()
b = df_small.groupby_fast("categorical").rolling(5, min_periods=1).sum(index_by_groups=True)
assert np.isclose(a, b).all()

901 ms ± 35.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
The slowest run took 5.00 times longer than the fastest. This could mean that an intermediate result is being cached.
128 ms ± 112 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
# 40x faster with output aligned to input
%timeit -n 1 df_small.groupby_fast("categorical").rolling(5, min_periods=1).sum();

The slowest run took 16.75 times longer than the fastest. This could mean that an intermediate result is being cached.
67 ms ± 112 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### EMAs - 10x / 70x faster

In [19]:
ema = df.groupby("categorical", observed=True).ewm(alpha=.5).mean()
ema_fast = df.groupby_fast("categorical").ema(alpha=.5, index_by_groups=True)
assert np.isclose(ema, ema_fast).all()

In [20]:
# > 10x faster with same behaviour
%timeit -n 1 df.groupby("categorical", observed=True).ewm(alpha=.5).mean()
%timeit -n 1 df.groupby_fast("categorical").ema(alpha=.5, index_by_groups=True)

4.37 s ± 480 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
327 ms ± 26.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Result aligned to input (like Polars) - 6-8x faster (80x vs Pandas)

In [21]:
%timeit -n 1 df.groupby_fast("categorical").ema(alpha=.5,)

70.8 ms ± 49.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit -n 1   
# YUCK
df_pl.select([
    c.ewm_mean(alpha=.5).over("categorical") for c in [polars.col.floats, polars.col.ints]
])

394 ms ± 43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
ema_gbl = df.groupby_fast("categorical").ema(alpha=.5,)
ema_pl = df_pl.select([
    col.ewm_mean(alpha=.5).over("categorical") for col in [polars.col.floats, polars.col.ints]
])
assert np.isclose(ema_gbl, ema_pl.to_pandas()).all()

#### quantiles

In [24]:
%timeit -n 1 df.groupby("ints", observed=True).median(numeric_only=True);
%timeit -n 1 df.groupby_fast("ints").median();

349 ms ± 18.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
290 ms ± 118 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%timeit -n 1 df.groupby("categorical", observed=True).median(numeric_only=True);
%timeit -n 1  df.groupby_fast("categorical").median();

563 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
159 ms ± 8.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Quantiles 6-10x Faster

In [36]:
q = [.25, .5, .75]
%timeit -n 1 df.groupby("ints", observed=True).quantile(q=q, numeric_only=True);
%timeit -n 1 df.groupby_fast("ints").quantile(q=q);
%timeit -n 1 GroupBy(df.ints).quantile(df.floats, q=q);

1.48 s ± 67.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
266 ms ± 7.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
267 ms ± 13.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit -n 1 df.groupby("categorical", observed=True).quantile(q=q, numeric_only=True);
%timeit -n 1 df.groupby_fast("categorical").quantile(q=q);

2.85 s ± 215 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
204 ms ± 4.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Faster crosstabs (4x/8x faster without/with margins)

In [30]:
# Without margins: 4x faster
%timeit -n 1 pd.crosstab(df.ints, df.categorical, df.floats, aggfunc="mean",)
%timeit -n 1 df.pivot_table("floats", "ints", "categorical", aggfunc="sum", observed=True)
%timeit -n 1 crosstab(df.ints, df.categorical, df.floats, aggfunc="mean", )

471 ms ± 84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
329 ms ± 6.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
120 ms ± 4.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
# With margins: 8x faster
%timeit -n 1 pd.crosstab(df.ints, df.categorical, df.floats, aggfunc="sum", margins=True)
%timeit -n 1 crosstab(df.ints, df.categorical, df.floats, aggfunc="sum", margins=True)

866 ms ± 56.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
119 ms ± 1.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
# With margins & mask: 8x faster
mask = df.floats > .2
%timeit -n 1 df.loc[mask].pivot_table("floats", "ints", "categorical", aggfunc="sum", margins=True, observed=True)
%timeit -n 1 crosstab(df.ints, df.categorical, df.floats, aggfunc="sum", margins=True, mask=mask)

722 ms ± 33.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
170 ms ± 24.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
